In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import matplotlib.image as mpimg

# import module we'll need to import our custom module
import shutil

# import all the custom modules in the libraries-file directory 
dest = "../working/"
lib_folder = "../input/libraries-file/"
src_files = os.listdir(lib_folder)
for file_name in src_files:
    full_file_name = os.path.join(lib_folder, file_name)
    if (os.path.isfile(full_file_name)):
        shutil.copy(full_file_name, dest)


['libraries-file', 'infofiles', 'full-keras-pretrained-no-top', 'humpback-whale-identification']


In [2]:
file = pd.read_csv('../input/humpback-whale-identification/train.csv')

In [3]:
import exploratory_analysis
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import trained_model as tm
import label_processor as lp
import image_processing as ip
import keras

Using TensorFlow backend.


In [4]:
from keras.layers import Activation
from keras.layers import Lambda
from keras.layers import Conv2D
from keras.layers import Add, UpSampling2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import ZeroPadding2D
from keras.layers import Input, Dense, Flatten
from keras.layers import Dropout, Concatenate
from keras.layers import BatchNormalization
from keras.models import Model

**For Neural Network**

Convert the data frame to a dictionary - to find the Id for the corresponding filename quickly

In [5]:
file_dict = dict(zip(file.Image,file.Id))

Create a dictionary with the Id as the key and the list [number_of_images_belong_to_that_ID, Index_assigned]

In [6]:
dict_unique_Ids = exploratory_analysis.find_uniq_values_counts(file, 'Id')

In [7]:
def network(pretrained_model_name, input_shape,output_shape):
    inp = Input(shape = input_shape)
    x = Lambda(lambda x:x)(inp)
    x = tm.pretrained_model(x,pretrained_model_name,(224,224,3)).output
    x = Flatten(name='Flatten')(x)
    x = Dense(10000, name = 'Fully_Connected', activation = 'relu')(x)
    x = BatchNormalization()(x)
    x = Dense(output_shape[1], activation = 'softmax')(x)
    
    model = Model(input = inp, output = x)
    
    return model

In [8]:
model = network('VGG16',(224,224,3),(1,5005))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [9]:
def generator_from_file():
    counter = 0
    for index in range(len(file['Image'])):

        image_name = file['Image'][index]
        path = '../input/humpback-whale-identification/train/'
        
        # Image reading and processing
        x = ip.conversion_resize_generator(color_conversion='Gray2RGB', filename_with_path=path+image_name, height=224, resize_='True',width=224)
                
        # Label creation and processing
        array_index = dict_unique_Ids[file_dict[image_name]][0]
        y = lp.label_generator(index=array_index,num_of_classes=5005)
        
        yield ([x],[y])

In [ ]:
optim=keras.optimizers.Adadelta(lr=0.8, rho=0.95, epsilon=None, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics = ['accuracy'])
import tensorflow as tf
steps_per_epoch = 100
with tf.device("/gpu:0"):
    model.fit_generator(
            generator_from_file(), steps_per_epoch=steps_per_epoch, shuffle=True, 
            epochs=30)

Trial